<a href="https://colab.research.google.com/github/dfvnunes/GTM-Setup/blob/main/Assist%C3%AAncias_por_Clube.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
!pip install openpyxl
!pip install xlsxwriter
import pandas as pd
import openpyxl
import xlsxwriter

df = pd.read_excel('/content/Por Clube.xlsx', sheet_name=None, thousands='.')

for sheet_name, sheet_df in df.items():
    # Identifica colunas Float64
    float64_cols = sheet_df.select_dtypes(include=['float64']).columns

    # Converte colunas Float64 para Int64
    if len(float64_cols) >0: # Only convert if there are float64 columns
      sheet_df[float64_cols] = sheet_df[float64_cols].astype('int64')

    # Atualiza as folhas no dicionário (ficheiro)
    df[sheet_name] = sheet_df

    # Exporta para Excel
with pd.ExcelWriter('Por Clube Int64.xlsx', engine='openpyxl') as writer:
    for sheet_name, sheet_df in df.items():
        sheet_df.to_excel(writer, sheet_name=sheet_name, index=False)

excel_file = pd.ExcelFile('/content/Por Clube Int64.xlsx')

# Dicionário com os nomes das ligas por época
ligas_por_epoca = {
    '2009-2010': {'Liga Sagres': 'Liga 1', 'Liga Vitalis': 'Liga 2'},
    '2010-2011': {'Liga Zon Sagres': 'Liga 1', 'Liga Orangina': 'Liga 2'},
    '2011-2012': {'Liga Zon Sagres': 'Liga 1', 'Liga Orangina' : 'Liga 2'},
    '2012-2013': {'Liga ZON Sagres': 'Liga 1', 'Segunda Liga': 'Liga 2'},
    '2013-2014': {'Liga ZON Sagres': 'Liga 1', 'Liga2 Cabovisão': 'Liga 2'},
    '2014-2015': {'Liga NOS': 'Liga 1', 'Segunda Liga': 'Liga 2'},
    '2015-2016': {'Liga NOS': 'Liga 1', 'Ledman LigaPRO': 'Liga 2'},
    '2016-2017': {'Liga NOS': 'Liga 1', 'Ledman LigaPRO': 'Liga 2'},
    '2017-2018': {'Liga NOS': 'Liga 1', 'Ledman LigaPRO': 'Liga 2'},
    '2018-2019': {'Liga NOS': 'Liga 1', 'Ledman LigaPRO': 'Liga 2'},
    '2019-2020': {'Liga NOS': 'Liga 1', 'LigaPro': 'Liga 2'},
    '2020-2021': {'Liga NOS': 'Liga 1', 'Liga Portugal SABSEG': 'Liga 2'},
    '2021-2022': {'Liga Portugal bwin': 'Liga 1', 'Liga Portugal SABSEG': 'Liga 2'},
    '2022-2023': {'Liga Portugal bwin': 'Liga 1', 'Liga Portugal SABSEG': 'Liga 2'},
    '2023-2024': {'Liga Portugal Betclic': 'Liga 1', 'Liga Portugal SABSEG': 'Liga 2'},
}

with pd.ExcelWriter('Por Clube Limpo.xlsx', mode='w') as writer:
    # Loop para iterar pelas folhas

        for sheet_name in excel_file.sheet_names:
         # Lê a folha atual
         df = excel_file.parse(sheet_name)

         # Aplica as funções ao DataFrame da folha
         if 'Ranking' in df.columns:
             df_drop = df.drop('Ranking', axis=1)
         else:
             df_drop = df.copy()

         # Substituir nomes das ligas de acordo com a época
         if sheet_name in ligas_por_epoca:
            substituicoes = ligas_por_epoca[sheet_name]
            df_drop['Competição'] = df_drop['Competição'].replace(substituicoes)



         styled_df = df_drop.style.format({'% média de ocupação': '{:.2%}'})
         # display(styled_df)
         df_drop['Temporada'] = sheet_name
         df_drop = df_drop[['Competição', 'Temporada', 'Clube', 'Nº jogos em casa', 'Espectadores na última jornada', 'Média espectadores jogo', 'Acumulado da época', '% média ocupação']]


         # Guarda o resultado na folha correspondente
         df_drop.to_excel(writer, sheet_name=sheet_name, index=False, header=True)
         worksheet = writer.sheets[sheet_name]

         for col_num, col_name in enumerate(df_drop.columns):

            # Lê a largura da coluna
            column_width = max(df_drop[col_name].astype(str).map(len).max(), len(col_name)) + 2


            # Define o alinhamento e a largura da coluna
            # Cria um formato para alinhamento
            cell_format = writer.book.add_format()

            # Define os parâmtros para o alinhamento das colunas
            cell_format.set_align('center')
            worksheet.set_column(col_num, col_num, column_width, cell_format)